In [1]:
import tensorflow as tf

print(tf.__version__)

2.8.0


In [2]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
#from keras.layers.embeddings import Embedding
from keras.layers import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
#from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [6]:
data = pd.read_csv('./dataset.csv')

data['task_name'] = data['task_name'].str.lower()
data['subtask_name'] = data['subtask_name'].str.lower()

In [7]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

In [8]:
def remove_stopwords(data):
  # data['task_name']=data['task_name'].fillna("")
  # data['subtask_name']=data['subtask_name'].fillna("")
  data['task_name without stopwords'] = data['task_name'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  data['subtask_name without stopwords'] = data['subtask_name'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result


In [9]:
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_task_name']= data_without_stopwords['task_name without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_task_name'] = data_without_stopwords['clean_task_name'].str.replace('[{}]'.format(string.punctuation), ' ')

data_without_stopwords['clean_subtask_name']= data_without_stopwords['subtask_name without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_subtask_name'] = data_without_stopwords['clean_subtask_name'].str.replace('[{}]'.format(string.punctuation), ' ')

C:\Users\r2fpdvzocm5ydw\AppData\Local\Temp\ipykernel_16620\1587466166.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data_without_stopwords['clean_task_name'] = data_without_stopwords['clean_task_name'].str.replace('[{}]'.format(string.punctuation), ' ')
C:\Users\r2fpdvzocm5ydw\AppData\Local\Temp\ipykernel_16620\1587466166.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  data_without_stopwords['clean_subtask_name'] = data_without_stopwords['clean_subtask_name'].str.replace('[{}]'.format(string.punctuation), ' ')


In [10]:
data_without_stopwords.head()

,Skill,Complexity,task_name,subtask_name,Duration,task_name without stopwords,subtask_name without stopwords,clean_task_name,clean_subtask_name
0,3,1,home page,api to validate and fetch hot products,3,home page,api validate fetch hot products,home page,api validate fetch hot products
1,2,0,home page,api to validate and fetch new products,2,home page,api validate fetch new products,home page,api validate fetch new products
2,3,1,categories page,api to validate and fetch all categories along...,3,categories page,api validate fetch categories along products b...,categories page,api validate fetch categories along products b...
3,2,0,categories page,api to validate and fetch a single category,2,categories page,api validate fetch single category,categories page,api validate fetch single category
4,2,0,categories page,api to filter and sort results,2,categories page,api filter sort results,categories page,api filter sort results


In [11]:

reviews = data_without_stopwords['clean_subtask_name']
reviews

0                       api validate fetch hot products
1                       api validate fetch new products
2     api validate fetch categories along products b...
3                    api validate fetch single category
4                               api filter sort results
5                      api validate fetch list products
6                               api filter sort results
7                       api persist chosen item db cart
8                   api persist chosen item db wishlist
9                    api validate fetch product details
10                      api persist chosen item db cart
11                  api persist chosen item db wishlist
12                                api authenticate user
13                        api validate fetch cart items
14                                api authenticate user
15    api validate fetch shipping address saved cust...
16                   api save validate customer address
17    api validate fetch city state names busine

In [12]:
reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])



In [13]:
sentiment = data_without_stopwords['Duration']

In [14]:
y = np.array(list(sentiment))

In [15]:
y

array([3, 2, 3, 2, 2, 2, 3, 3, 3, 1, 3, 3, 4, 4, 4, 2, 2, 2, 4, 2, 4, 2,
       4, 2, 2, 2, 2, 4, 3, 3, 4, 3, 4, 3, 4, 3, 4, 5, 4, 3])

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)
X_test

['api validate fetch list products',
 'api validate fetch cart items',
 'api persist chosen item db cart',
 'api validate fetch hot products',
 'api validate view  add change billing method',
 'api authenticate user',
 'api validate fetch product details',
 'api validate fetch delivery methods supported business']

In [17]:
len(Y_train)

32

In [18]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [19]:
words_to_index = tokenizer.word_index
for key, value in words_to_index.items():
    words_to_index[key] = words_to_index[key] - 1
words_to_index

{'api': 0,
 'validate': 1,
 'fetch': 2,
 'authenticate': 3,
 'user': 4,
 'persist': 5,
 'db': 6,
 'wishlist': 7,
 'address': 8,
 'chosen': 9,
 'item': 10,
 'profile': 11,
 'customer': 12,
 'cart': 13,
 'order': 14,
 'shipping': 15,
 'orders': 16,
 'cancelled': 17,
 'filter': 18,
 'sort': 19,
 'results': 20,
 'business': 21,
 'products': 22,
 'category': 23,
 'payment': 24,
 'view': 25,
 'change': 26,
 'settings': 27,
 'save': 28,
 'details': 29,
 'pending': 30,
 'completed': 31,
 'city': 32,
 'state': 33,
 'names': 34,
 'operates': 35,
 'categories': 36,
 'along': 37,
 'belong': 38,
 'send': 39,
 'information': 40,
 'server': 41,
 'approval': 42,
 'response': 43,
 'either': 44,
 'success': 45,
 'method': 46,
 'items': 47,
 'delivery': 48,
 'new': 49,
 'saved': 50,
 'if': 51,
 'exists': 52,
 'methods': 53,
 'supported': 54,
 'apis': 55,
 'show': 56,
 'stats': 57,
 'profit': 58,
 'sales': 59,
 'etc': 60,
 'single': 61}

In [20]:
len(words_to_index)

62

In [21]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [22]:
word_to_vec_map = read_glove_vector('./glove.6B.50d.txt')

In [23]:
maxLen = 150

In [24]:

vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [55]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1)(X)
  print("X: ")
  print( X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [56]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='softmax')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='softmax')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='softmax')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='softmax')(X)
  X = Dense(1)(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [57]:
model = Sequential()
model.add(LSTM(32, input_shape=(None, 10)))
model.add(Dense(1))
# model.summary()

X: 
KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='dense_6/BiasAdd:0', description="created by layer 'dense_6'")


In [58]:
model_1d = conv1d_model((maxLen,))
# model_1d.summary()

In [59]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [60]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(32, 150)

In [61]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0100)
model_1d.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

In [62]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
1/1 [==============================] - 4s 4s/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 0s 326ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 3/15
1/1 [==============================] - 0s 202ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 4/15
1/1 [==============================] - 0s 195ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 5/15
1/1 [==============================] - 0s 234ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 6/15
1/1 [==============================] - 0s 204ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 7/15
1/1 [==============================] - 0s 203ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 8/15
1/1 [==============================] - 0s 225ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 9/15
1/1 [==============================] - 0s 224ms/step - loss: 45.7928 - accuracy: 0.0000e+00
Epoch 10/15
1/1 [==============================] - 0s 188ms/step - loss: 45.

In [78]:
adam = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='mean_squared_error', metrics=['accuracy'])

In [79]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
1/1 [==============================] - 20s 20s/step - loss: 7.7056 - accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 2s 2s/step - loss: 7.3784 - accuracy: 0.0000e+00
Epoch 3/15
1/1 [==============================] - 3s 3s/step - loss: 6.8968 - accuracy: 0.0000e+00
Epoch 4/15
1/1 [==============================] - 1s 1s/step - loss: 6.5394 - accuracy: 0.0000e+00
Epoch 5/15
1/1 [==============================] - 1s 1s/step - loss: 6.1309 - accuracy: 0.0000e+00
Epoch 6/15
1/1 [==============================] - 1s 1s/step - loss: 5.7375 - accuracy: 0.0000e+00
Epoch 7/15
1/1 [==============================] - 2s 2s/step - loss: 5.3245 - accuracy: 0.0000e+00
Epoch 8/15
1/1 [==============================] - 2s 2s/step - loss: 4.9287 - accuracy: 0.0000e+00
Epoch 9/15
1/1 [==============================] - 2s 2s/step - loss: 4.5062 - accuracy: 0.0000e+00
Epoch 10/15
1/1 [==============================] - 2s 2s/step - loss: 4.1931 - accuracy: 0.0000e+00
Epoch 1

In [80]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [81]:
model.evaluate(X_test_indices, Y_test)

1/1 [==============================] - 6s 6s/step - loss: 2.5989 - accuracy: 0.1250


[2.598870277404785, 0.125]

In [82]:
model_1d.evaluate(X_test_indices, Y_test)

1/1 [==============================] - 0s 148ms/step - loss: 46.2748 - accuracy: 0.0000e+00


[46.274845123291016, 0.0]

In [83]:
preds = model_1d.predict(X_test_indices)

In [84]:
n = np.random.randint(0,len(X_test))

for i in range(n):
    print(f'Subtask name: {X_test[i]}, predicted duration : {preds[i]}, correct duration : {Y_test[i]}')


Subtask name: api validate fetch list products, predicted duration : [-0.01109354], correct duration : 2
Subtask name: api validate fetch cart items, predicted duration : [-0.01109354], correct duration : 4
Subtask name: api persist chosen item db cart, predicted duration : [-0.01109354], correct duration : 3


In [85]:
# print(f'predicted duration : {preds[n]}')
# print(f'correct duration : {Y_test[n]}')

In [86]:
preds[n]

array([-0.01109354], dtype=float32)

In [87]:
Y_test[n]

3

In [88]:
model_1d.save_weights('./imdb_weights_con1vd.hdf5')

In [89]:
def add_score_predictions(data):

  data['duration score'] = 0

  reviews_list_idx = pad_sequences(reviews_list_idx, maxlen=maxLen, padding='post')

  review_preds = model.predict(reviews_list_idx)

  data['duration score'] = review_preds

  pred_sentiment = np.array(list(map(lambda x : 'positive' if x > 0.5 else 'negative',review_preds)))

  data['predicted duration'] = 0

  data['predicted duration'] = pred_sentiment

  return data

In [90]:
# data = add_score_predictions(data, reviews_list_idx)

In [91]:
data

,Skill,Complexity,task_name,subtask_name,Duration,task_name without stopwords,subtask_name without stopwords,clean_task_name,clean_subtask_name
0,3,1,home page,api to validate and fetch hot products,3,home page,api validate fetch hot products,home page,api validate fetch hot products
1,2,0,home page,api to validate and fetch new products,2,home page,api validate fetch new products,home page,api validate fetch new products
2,3,1,categories page,api to validate and fetch all categories along...,3,categories page,api validate fetch categories along products b...,categories page,api validate fetch categories along products b...
3,2,0,categories page,api to validate and fetch a single category,2,categories page,api validate fetch single category,categories page,api validate fetch single category
4,2,0,categories page,api to filter and sort results,2,categories page,api filter sort results,categories page,api filter sort results
5,2,0,plp,api to validate and fetch list of products,2,plp,api validate fetch list products,plp,api validate fetch list products
6,3,1,plp,api to filter and sort results,3,plp,api filter sort results,plp,api filter sort results
7,3,1,plp,api to persist the chosen item in db for cart,3,plp,api persist chosen item db cart,plp,api persist chosen item db cart
8,3,1,plp,api to persist the chosen item in db for wishlist,3,plp,api persist chosen item db wishlist,plp,api persist chosen item db wishlist
9,1,0,pdp,api to validate and fetch product details,1,pdp,api validate fetch product details,pdp,api validate fetch product details
